### **■ 유통 판매량 예측 및 재고 최적화**
# **단계 3: 모델링 및 비즈니스 평가**

<img src = "https://github.com/Jangrae/img/blob/master/store.png?raw=true" width=800, align="left"/>

# **⏰ 수행 과제**

다음과 같은 과정으로 프로젝트를 진행합니다.

#### **1. 환경 설정**
- 이후 진행에 필요한 환경 설정을 수행합니다.

#### **2. 4차 모델링**
- Random Forest, LigntGBM 알고리즘으로 모델링합니다.
- 이번에는 하이퍼파라미터 튜닝을 통해 모델의 성능을 높여봅니다.
- 변화되는 모델의 성능을 기록합니다.

#### **3. 파이프라인 구축**
- 새로 읽어온 데이터에 대해 모델이 예측핳 수 있는 형태의 데이터 셋을 만들어야 합니다.
- 이러한 데이터 셋을 만드는 파이프라인을 함수를 만들어 보세요.

#### **4. 최종 평가**
- 새로운 데이터에 대해 예측하고 성능을 평가합니다.
- 재고량을 평가하는 다음 함수를 활용해 비즈니스 관점의 평가를 수행하세요.
- 최종 평가 결과를 기록하고 비교합니다.

# **1. 환경 설정**

- 이후 진행에 필요한 환경 설정을 수행합니다.

## **(1) 경로 설정**

- 프로젝트 수행 환경에 맞게 파일 경로를 설정합니다.

### **1) 로컬 수행(Anaconda)**
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# 기본 경로
path = ''

### **2) 구글 콜랩 수행**

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [ ]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

## **(2) 라이브러리 불러오기**

- 이후 사용할 라이브러리를 모두 불러옵니다.

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

- 모델이 예측한 결과를 시각화할 때 다음 함수를 활용합니다.

In [ ]:
# 함수 만들기
def plot_model_result(y_train, y_val, y_pred):
    
    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    y_pred = pd.Series(y_pred.reshape(-1,), index=y_val.index)

    # 전체 시각화
    plt.figure(figsize=(12, 5))
    plt.subplot(2,1,1)
    plt.plot(y_train, label='y_train', color='tab:blue')
    plt.plot(y_val, label='y_val', color='tab:green')
    plt.plot(y_pred, label='y_pred', color='tab:orange')
    plt.legend()
    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'y_val', color='tab:green')
    plt.plot(y_pred, label = 'y_pred', color='tab:orange')
    plt.legend()
    plt.tight_layout()
    plt.show()

## **(3) 데이터 불러오기**

- 이후 분석 대상이 되는 파일을 불러오고 기본 정보를 확인합니다.

### **1) 데이터 불러오기**

- 이전 과정에서 저장한 pkl 파일을 불러옵니다.
- 데이터프레임 이름은 data03, data12, data42가 되게 합니다.
- 다음과 같은 형태의 구문으로 불러옵니다.
~~~
mydata = joblib.load(path + 'mydata.pkl')
~~~

In [ ]:
# 데이터 불러오기
data03 = joblib.load(path + 'data03.pkl')
data12 = joblib.load(path + 'data12.pkl')
data42 = joblib.load(path + 'data42.pkl')

### **2) 기본 정보 확인**

- 각 데이터의 기본 정보를 확인합니다.

In [ ]:
# 3번 상품
data03.head()

In [ ]:
# 12번 상품
data12.head()

In [ ]:
# 42번 상품
data42.head()

# **2. 4차 모델링**

- Random Forest, LigntGBM 알고리즘으로 모델링합니다.
- 모델링 과정에서 변수 이름은 **x_train, x_val, y_train, y_val, y_pred**을 사용합니다.
- 이번에는 **하이퍼파라미터 튜닝**을 통해 모델의 성능을 높여봅니다.
- **검증용 데이터**는 학습용 데이터에서 **최근 120일간**의 데이터를 사용합니다.
- 변화되는 모델의 성능을 기록합니다.

In [ ]:
# 함수 만들기
def preproc(data):
    
    # x, y 분리
    target = 'Target'
    x = data.drop(['Date', target], axis=1)
    y = data.loc[:, target]

    # 가변수화
    weekdays = ['Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    x['WeekDay'] = pd.Categorical(x['WeekDay'], categories=weekdays)
    x['Month'] = pd.Categorical(x['Month'], categories=months)
    x = pd.get_dummies(x, columns=['WeekDay', 'Month'], dtype=int)
    
    # 학습용, 검증용 분할
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=120, shuffle=False)

    return x_train, x_val, y_train, y_val

## **(1) Random Forest 모델**

### **1) 상품: 3 - Beverage**

- GridSearchCV를 사용해 3번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model03_rdf**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data03)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model03_rdf = GridSearchCV(RandomForestRegressor(),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model03_rdf.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model03_rdf.best_params_)
print(model03_rdf.best_score_)

# 예측하기
y_pred = model03_rdf.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

### **2) 상품: 12 - Milk**

- GridSearchCV를 사용해 12번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model12_rdf**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data12)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model12_rdf = GridSearchCV(RandomForestRegressor(),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model12_rdf.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model12_rdf.best_params_)
print(model12_rdf.best_score_)

# 예측하기
y_pred = model12_rdf.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

### **3) 상품: 42 - Agricultural products**

- GridSearchCV를 사용해 42번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model42_rdf**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data42)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model42_rdf = GridSearchCV(RandomForestRegressor(),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model42_rdf.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model42_rdf.best_params_)
print(model42_rdf.best_score_)

# 예측하기
y_pred = model42_rdf.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

## **(2) LightGBM 모델**

### **1) 상품: 3 - Beverage**

- GridSearchCV를 사용해 3번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model03_lgb**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data03)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model03_lgb = GridSearchCV(LGBMRegressor(verbose=-1),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model03_lgb.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model03_lgb.best_params_)
print(model03_lgb.best_score_)

# 예측하기
y_pred = model03_lgb.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

### **2) 상품: 12 - Milk**

- GridSearchCV를 사용해 12번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model12_lgb**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data12)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model12_lgb = GridSearchCV(LGBMRegressor(verbose=-1),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model12_lgb.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model12_lgb.best_params_)
print(model12_lgb.best_score_)

# 예측하기
y_pred = model12_lgb.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

### **3) 상품: 42 - Agricultural products**

- GridSearchCV를 사용해 42번 상품에 대해 모델링하고 성능을 확인합니다..
- 모델 이름은 **model42_lgb**로 합니다.

In [ ]:
# 데이터 준비
x_train, x_val, y_train, y_val = preproc(data42)

# 파라미터 범위
params = {'max_depth': range(1, 21)}

# 선언하기
model42_lgb = GridSearchCV(LGBMRegressor(verbose=-1),
                           params,
                           cv=5,
                           n_jobs=-1)

# 학습하기
model42_lgb.fit(x_train, y_train)

# 파라미터, 예측 성능 확인
print(model42_lgb.best_params_)
print(model42_lgb.best_score_)

# 예측하기
y_pred = model42_lgb.predict(x_val)

# 평가하기
print('RMSE:', mean_squared_error(y_val, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_val, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_val, y_pred))
print('R2:', r2_score(y_val, y_pred))

- 모델이 예측한 결과를 시각화해 확인합니다.

In [ ]:
# 예측 결과 시각화
plot_model_result(y_train, y_val, y_pred)

# **3. 파이프라인 구축**

- 새로 읽어온 데이터에 대해 모델이 예측핳 수 있는 형태의 데이터 셋을 만들어야 합니다.
- 이러한 데이터 셋을 만드는 파이프라인을 함수를 만들어 보세요.

## **(1) 새로운 데이터 불러오기**

- 평가에 사용할 새로운 데이터를 불러옵니다.
- 데이터프레임 이름은 다음과 같이 통일합니다.
    - sales: 판매 정보
    - orders: 고객 방문수
    - oil_price: 휘발유 가격
    - stores: 매장 정보
    - products: 상품 정보

In [ ]:
# 데이터 불러오기
sales = pd.read_csv(path + 'sales_test.csv')
orders = pd.read_csv(path + 'orders_test.csv')
oil_price = pd.read_csv(path + 'oil_price_test.csv')
stores = pd.read_csv(path + 'stores.csv')
products = pd.read_csv(path + 'products.csv')

In [ ]:
# datetime 형으로 변환
sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )

## **(2) 데이터 파이프라인 구축**

- 다음과 같은 처리를 수행하는 파이프라인이 되어야 합니다.
    - 입력: sales_data, orders_data, oil_price_data, Product_ID
    - 처리:
        - 머신러닝 모델에 전달할 수 있는, 즉 x_train, x_val과 동일한 형태의 데이터 셋 만들기
        - 결측치 처리, 가변수화, 변수 제거, 변수 추가 등 일괄 처리
    - 출력: 전처리 완료된 데이터 프레임
    

In [ ]:
# 함수 만들기
def make_dataset(Product_ID, sales, orders, oil_price, products, stores):

    # 1. 기본 데이터 준비
    leadtime = products.loc[products['Product_ID']==Product_ID, 'LeadTime'].values[0]    
    temp1 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID']==Product_ID), ['Date', 'Qty']]
    temp2 = orders.loc[orders['Store_ID']==44, ['Date', 'CustomerCount']]
    temp3 = pd.merge(temp1, temp2, on='Date', how='left')

    # 2. Feature Engineering
    # 날짜 요소 추출
    temp3['WeekDay'] = temp3['Date'].dt.day_name()
    temp3['Month'] = temp3['Date'].dt.month

    # Oil Price
    temp3 = pd.merge(temp3, oil_price, on='Date', how='left')
    temp3['WTI_Price'] = temp3['WTI_Price'].rolling(14, min_periods=1).mean()

    # 동일 카테고리 판매량 합계
    Category = products.loc[products['Product_ID'].isin([Product_ID]), 'Category'].to_list()
    Product_IDs = products.loc[products['Category'].isin(Category), 'Product_ID'].to_list()
    temp4 = sales.loc[(sales['Store_ID']==44) & (sales['Product_ID'].isin(Product_IDs))]
    temp4 = temp4.groupby(by='Date', as_index=False)['Qty'].sum()
    temp4.columns = ['Date', 'Category_Qty']
    temp3 = pd.merge(temp3, temp4, on='Date', how='left')

    # 동일 지역 방문객 수
    City = stores.loc[stores['Store_ID']==44, 'City'].values[0]
    Store_IDs = stores.loc[stores['City']==City, 'Store_ID'].to_list()
    temp5 = orders.loc[orders['Store_ID'].isin(Store_IDs)]
    temp5 = temp5.groupby('Date', as_index=False)['CustomerCount'].sum()
    temp5.columns = ['Date', 'City_CustCount']
    temp3 = pd.merge(temp3, temp5, on='Date', how='left')

    # 결측치 처리
    temp3.interpolate(method='linear', inplace=True)
    temp3.fillna(method='bfill', inplace=True)
    
    # 1일 전 판매량
    temp3['Qty_Lag_1'] =  temp3['Qty'].shift(1)

    # 2일 전 판매량
    temp3['Qty_Lag_2'] =  temp3['Qty'].shift(2)

    # 7일 전 판매량
    temp3['Qty_Lag_7'] =  temp3['Qty'].shift(7)

    # 최근 7일 판매량 평균
    temp3['Qty_Lag_7_mean'] = temp3['Qty'].rolling(7, min_periods=1).mean()

    # 최근 14일 판매량 평균
    temp3['Qty_Lag_14_mean'] = temp3['Qty'].rolling(14, min_periods=1).mean()

    # 3. Target 추가
    temp3['Target'] = temp3['Qty'].shift(-leadtime)
    
    # 결측치 처리
    temp3.dropna(inplace=True)
    temp3.reset_index(drop=True, inplace=True)
    
    return temp3

In [ ]:
# 함수 만들기
def preproc_newdata(data):
    
    # x, y 분리
    target = 'Target'
    x = data.drop(['Date', target], axis=1)
    y = data.loc[:, target]

    # 가변수화
    weekdays = ['Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday']
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    x['WeekDay'] = pd.Categorical(x['WeekDay'], categories=weekdays)
    x['Month'] = pd.Categorical(x['Month'], categories=months)
    x = pd.get_dummies(x, columns=['WeekDay', 'Month'], dtype=int)

    return x, y

In [ ]:
def data_pipeline(Product_ID, sales, orders, oil_price, products, stores):
    
    # 기본 데이터셋 만들기
    ds = make_dataset(Product_ID, sales, orders, oil_price, products, stores)

    # 모델링을 위한 구조 만들기
    x, y = preproc_newdata(ds)

    # 결과 리턴
    return x, y

# **4. 최종 평가**

- 새로운 데이터에 대해 예측하고 성능을 평가합니다.
- 재고금액을 평가하는 다음 함수를 활용해 비즈니스 관점의 평가를 수행하세요.
- 최종 평가 결과를 기록하고 비교합니다.

In [ ]:
# 재고량 평가 함수 만들기
def inv_simulator(y, pred, safe_stock, price):
    
    # 시뮬레이션 df 틀 만들기
    temp = pd.DataFrame({'y': y.reshape(-1,), 'pred': pred.reshape(-1,).round()})
    temp['y'] = temp['y'].astype(int)
    temp['pred'] = temp['pred'].astype(int)

    temp['base_stock'] = 0
    temp['close_stock'] = 0
    temp['order'] = 0
    temp['receive'] = 0

    # 시뮬레이션
    for i in range(len(temp)-2):  # 발주량은 2일 후 판매 예측량에 기초하므로 계산을 위해 마지막 2개 행 제외
        if i == 0:  #첫 행, 2일 전 데이터가 없으므로
            temp.loc[i, 'receive'] = temp.loc[i, 'y']  # 입고량은 실판매량으로 계산
            temp.loc[i, 'base_stock'] = temp.loc[i, 'receive'] + safe_stock  # 기초재고는 실판매량 + 안전재고로 계산

        elif i == 1: # 2일 전 행, 2일 전 데이터가 없음
            temp.loc[i, 'receive'] = temp.loc[i, 'y'] # 입고량은 실판매량으로 계산
            temp.loc[i, 'base_stock'] = temp.loc[i,'receive'] + temp.loc[i-1, 'close_stock']
        else:        # 나머지 전체 행
            temp.loc[i, 'receive'] = temp.loc[i-2,'order']    # 입고량 = 2일 전 발주량
            temp.loc[i, 'base_stock'] = temp.loc[i, 'receive']+temp.loc[i-1, 'close_stock']  # 기초재고 = 입고량 + 전날 기말재고

        # 기말재고 = 기초재고 - 판매량, 만약 0보다 작으면 0으로
        stock = round(temp.loc[i, 'base_stock'] - temp.loc[i, 'y'])
        temp.loc[i, 'close_stock'] = np.where(stock > 0, stock, 0)

        # 발주량 = 2일후 판매예측량 + 안전재고 - 기말재고,  만약 주문량이 0보다 작으면 0
        order = temp.loc[i+2, 'pred'] + safe_stock - temp.loc[i, 'close_stock']
        temp.loc[i, 'order'] = np.where(order>0, order, 0)

    # 기회손실 = 만약 (기초재고 - 실판매량)이 0보다 작으면, 그만큼이 기회손실
    temp['lost'] = np.where((temp['base_stock']-temp['y']) < 0, (temp['base_stock']-temp['y']), 0).round()

    inventory = temp[:len(temp) - 2]

    # 측정지표 계산
    DailyStock = ((inventory['base_stock'] + inventory['close_stock']) / 2)

    AvgDailyStock = round(DailyStock.mean(), 3)
    AvgDailyStockAmt = AvgDailyStock * price
    lost_sum = inventory['lost'].sum()

    print(f'* 일평균 재고량: {AvgDailyStock:.2f}')
    print(f'* 일평균 재고금액: {AvgDailyStockAmt:.2f}')
    print(f'* 기회손실 수량: {lost_sum}')

    return inventory

## **(1) 상품: 3 - Beverage**

- 3번 상품에 대해 모델 성능을 평가하고 기록합니다.

### **1) Random Forest 모델**

- Random Forest 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(3, sales, orders, oil_price, products, stores)
y_pred = model03_rdf.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 14130, 8)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock',  'order', 'lost']])
plt.legend(['y', 'pred', 'close_stock', 'order', 'lost'])
plt.show()

In [ ]:
result

### **2) LightGBM 모델**

- LightGBM 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(3, sales, orders, oil_price, products, stores)
y_pred = model03_lgb.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 15018, 8)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock', 'lost', 'order']])
plt.legend(['y', 'pred', 'close_stock', 'lost', 'order'])
plt.show()

## **(2) 상품: 12 - Milk**

- 12번 상품에 대해 모델 성능을 평가하고 기록합니다.

### **1) Random Forest 모델**

- Random Forest 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(12, sales, orders, oil_price, products, stores)
y_pred = model12_rdf.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 9319, 6)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock', 'lost', 'order']])
plt.legend(['y', 'pred', 'close_stock', 'lost', 'order'])
plt.show()

### **2) LightGBM 모델**

- LightGBM 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(12, sales, orders, oil_price, products, stores)
y_pred = model12_lgb.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 12990, 6)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock', 'lost', 'order']])
plt.legend(['y', 'pred', 'close_stock', 'lost', 'order'])
plt.show()

## **(3) 상품: 42 - Agricultural products**

- 42번 상품에 대해 모델 성능을 평가하고 기록합니다.

### **1) Random Forest 모델**

- Random Forest 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(42, sales, orders, oil_price, products, stores)
y_pred = model42_rdf.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 31, 5)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock', 'lost', 'order']])
plt.legend(['y', 'pred', 'close_stock', 'lost', 'order'])
plt.show()

### **2) LightGBM 모델**

- LightGBM 모델로 예측하고 평가합니다.

In [ ]:
# 예측하기
x_test, y_test = data_pipeline(42, sales, orders, oil_price, products, stores)
y_pred = model42_lgb.predict(x_test)

# 평가하기
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

In [ ]:
# 비즈니스 평가
result = inv_simulator(y_test.values, y_pred, 23, 5)

plt.figure(figsize=(8, 4))
plt.plot(result[['y', 'pred', 'close_stock', 'lost', 'order']])
plt.legend(['y', 'pred', 'close_stock', 'lost', 'order'])
plt.show()